In [1]:
import sys

sys.path.insert(0, '..')

import shutil
import os
import uuid
from biosimulator_processes.execute import exec_utc_comparison
from verification_service.worker.compare import generate_biosimulators_utc_comparison
from verification_service.data_model import MongoDbConnector
from pymongo.mongo_client import MongoClient
from dotenv import load_dotenv
import os 

_outs = './test_outputs'
if os.path.exists(_outs):
    shutil.rmtree(_outs)


def jobid(): return str(uuid.uuid4())


load_dotenv("../verification_service/.env")
uri = os.getenv("MONGO_DB_URI")
omex_source_dir = './examples/sbml-core'
omex_name = 'Elowitz-Nature-2000-Repressilator.omex'
omex_fp = os.path.join(omex_source_dir, omex_name)
out_dir = './test_outputs'
simulators = ['amici', 'copasi', 'tellurium']
spec_name = 'cI mRNA'
job_id = jobid()
db_connector = MongoDbConnector(client=MongoClient(uri), database_id="service_requests")

Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Cannot register SimpleNeuron. Error:
**
No module named 'pyneuroml'
**


In [2]:
from verification_service.worker.main import check_jobs

result = await check_jobs()

result

In progress job already exists: {'_id': ObjectId('66766ee1adc374bf1b782d0b'), 'job_id': 'eea465cc-27a8-47e5-96a2-3710d06cf0fc', 'status': 'IN_PROGRESS', 'timestamp': '2024-06-22 06:27:45.632271', 'comparison_id': 'test-comparison-0'}
Sleeping for 5...zzzzzzz...
In progress job already exists: {'_id': ObjectId('66766ee6adc374bf1b782d0d'), 'job_id': '7b1be727-15e2-452c-990f-356c2a215cfe', 'status': 'IN_PROGRESS', 'timestamp': '2024-06-22 06:27:50.804118', 'comparison_id': 'test-comparison-1'}
Sleeping for 5...zzzzzzz...
In progress job already exists: {'_id': ObjectId('66766eebadc374bf1b782d0f'), 'job_id': 'a87ac4bf-c439-49f7-84fe-b4c178f78ddb', 'status': 'IN_PROGRESS', 'timestamp': '2024-06-22 06:27:55.953769', 'comparison_id': 'test-comparison-3'}
Sleeping for 5...zzzzzzz...
In progress job already exists: {'_id': ObjectId('66766ef1adc374bf1b782d11'), 'job_id': '07b607be-b55f-45ad-afad-a5c77dae6b02', 'status': 'IN_PROGRESS', 'timestamp': '2024-06-22 06:28:01.127150', 'comparison_id': '

{'status': 'all jobs completed.'}